In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage, StickerSendMessage
from IPython.display import display
from IPython.display import Markdown
from datetime import datetime
import google.generativeai as genai
import json
import configparser
import PIL.Image
import sqlite3
global user_states


config = configparser.ConfigParser()
config.read('config.ini')

line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token'))
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))
genai.configure(api_key=config.get('line-bot', 'google_api_key'))

app = Flask(__name__)

# channel 4:判斷食物 5:使用者校正 1:英文翻譯 2:日文翻譯 0:一般機器人對話
user_states = {}

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    json_data_show = json.dumps(json_data, indent=4) #格式不同，不可混用
    print(json_data_show) # 印出 json_data
    global chat
    global foodRecord,translate_en,translate_jp
    global channel,tempfood
    class adjustform:
        def __init__(self, text):
            self.text = text
    
    try:
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        tk = json_data['events'][0]['replyToken']       # 取得 reply token
        mtype = json_data['events'][0]['message'] ['type']
        userId = json_data['events'][0]['source']['userId'] #取得使用者的id
        
        
        if userId not in user_states:
            print("創建新使用者")
            user_states[userId] = {
                'channel': None,
                'chat': None,
                'foodRecord': None,
                'translate_en': None,
                'translate_jp': None,
                'tempfood': None,
                'imagejudge': None }
        
        user_state = user_states[userId]
        print(user_state)
        
        if(mtype == 'text'):
            
            msg = json_data['events'][0]['message']['text']    # 取得使用者發送的訊息
            
            if(msg == "我想跟你聊天~"):
                model = genai.GenerativeModel('gemini-pro')
                user_state['chat'] = model.start_chat(history=[])
                
                user_state['channel'] = 0
                
                response = user_state['chat'].send_message('嗨你好!我想跟你聊天!!，接下來請使用繁體中文與我對話')
                print(user_state)
            elif(msg == "開始記錄飲食~"):
                model = genai.GenerativeModel('gemini-pro')
                user_state['foodRecord'] = model.start_chat(history=[])
                print(msg)
                response = adjustform(text='請開始上傳您的飲食圖片!!')
                a = user_state['foodRecord'].send_message('請記住待會我跟你說的食物')
                print(a)
            elif(msg == "產生飲食報告~"):
                user_state['channel'] = 6
                print(user_state['foodRecord'])
                return 'OK'
            elif(msg == "是!" and user_state['channel'] == 4):
                print("進入正確判斷")
                recordResponse = user_state['foodRecord'].send_message(user_state['tempfood'])
                print(recordResponse.text)
                response = adjustform(text='已完成記錄!!請繼續上傳飲食圖片~')
            elif(msg == "否!" and user_state['channel'] == 4):
                print("進入錯誤判斷")
                user_state['channel'] = 5
                response = adjustform(text='請輸入您攝取的食物名稱及份量')
            elif(user_state['channel'] ==5):
                print("進入使用者輸入判斷")
                model = genai.GenerativeModel('gemini-pro')
                askquestion = "請告訴我"+ msg +"的熱量及營養成分"
                print(askquestion)
                reconfirm = model.generate_content(askquestion)
                print(reconfirm.text)
                recordResponse = user_state['foodRecord'].send_message(reconfirm.text)
                print(recordResponse.text)
                
                response = adjustform(text='已完成記錄!!')
            elif(msg == "記錄今日體重"):
                print('enter record')
                response = adjustform(text='請輸入您的今日體重! e.g.80')
                user_state['channel'] = 7
            elif(msg == "查詢過往體重"):
                print('enter search')
                response = adjustform(text='請輸入您想查詢的日期! \n e.g.2024-05-12')
                user_state['channel'] = 8
            elif(msg == "英文小幫手"):
                model = genai.GenerativeModel('gemini-pro')
                user_state['translate_en'] = model.start_chat(history=[])
                user_state['channel'] = 1
                response = adjustform(text='請輸入你想翻譯的東東!!')
                b = user_state['translate_en'].send_message('請把我傳送的單字或句子翻譯成英文回傳')
                print(b)
            elif(msg == "日文小幫手"):
                model = genai.GenerativeModel('gemini-pro')
                user_state['translate_jp'] = model.start_chat(history=[])
                user_state['channel'] = 2
                response = adjustform(text='請輸入你想翻譯的東東!!')
                c = user_state['translate_jp'].send_message('請把我傳送的單字或句子翻譯成日文回傳')
                print(c)
            elif(msg == "請告訴我如何使用~"):
                print("不回傳東西，使用自動回復")
                return 'OK'
            elif(msg == "還要更多"):
                print("不回傳東西，使用自動回復")
                return 'OK'
            elif(user_state['channel'] == 6):
                askquestion_2 = '請把我剛剛傳給你的食物用文字整理，並給我今日攝取的總熱量，我的飲食目標是'+msg+'，請根據目標給我飲食建議'
                print(askquestion_2)
                response = user_state['foodRecord'].send_message(askquestion_2)
            elif(user_state['channel'] == 7):
                dbfile = "database.db"
                conn = sqlite3.connect(dbfile)
                current_date = datetime.now().date()
                print(userId,msg,current_date)
                commend = f"INSERT INTO record (user,weight,date) VALUES ('{userId}','{msg}','{current_date}');"
                print(commend)
                conn.execute(commend)
                conn.commit()
                conn.close()
                response = adjustform(text=f'記錄成功!! \n {current_date} 的體重為 {msg}')
            elif(user_state['channel'] == 8 ):
                dbfile = "database.db"
                conn = sqlite3.connect(dbfile)
                cursor = conn.cursor()
        
                cursor.execute("SELECT weight FROM record WHERE date=? AND user=?", (msg,userId))
                result = cursor.fetchone()
    
                conn.close()
                
                if result:
                    weight = result[0]
                    response = adjustform(text=f'您 {msg} 的體重為 {weight} !!')
                else:
                    response = adjustform(text=f'沒有{msg} 的體重記錄!!')
            else:
                if(user_state['channel'] == 0):
                    print('進入迴圈')
                    response = user_state['chat'].send_message(msg)
                    print(user_state['chat'])
                if(user_state['channel'] == 1):
                    print('進入英文翻譯')
                    response = user_state['translate_en'].send_message(msg)
                if(user_state['channel'] == 2):
                    print('進入日文翻譯')
                    response = user_state['translate_jp'].send_message(msg)
                    
            reply = TextSendMessage(text=response.text)
            
        elif(mtype == 'sticker'):
            print("收到的是貼圖")
            sticker_message = StickerSendMessage(sticker_id=52002735, package_id=11537)
            line_bot_api.reply_message(tk,sticker_message)
            
            return 'OK'
            
        elif(mtype == 'image'):
            msgID = json_data['events'][0]['message']['id']
            message_content = line_bot_api.get_message_content(msgID)
            
            with open(f'{msgID}.jpg', 'wb') as fd:
                fd.write(message_content.content)# 以二進位的方式寫入檔案
            print('成功寫入圖片')
            
            user_state['channel'] = 4
            user_state['imagejudge'] = genai.GenerativeModel('gemini-pro-vision')
            img = PIL.Image.open(f'{msgID}.jpg')
            
            response = user_state['imagejudge'].generate_content(["請告訴我圖片中的食物名稱，並簡述它的熱量及營養成分，營養成分必須包含熱量、蛋白質含量、脂肪含量、碳水化合含量.", img])
            print('成功獲得回應')
            checkreply = "請問您吃的東西是否為:"+'\n\n'+response.text
            print(checkreply)
            reply = TextSendMessage(text=checkreply)
            
            print(reply)
            user_state['tempfood'] = response.text
#             detail = detail+"請幫我記住我今天吃了這樣食物"
            
        else:
            print('其他')

        line_bot_api.reply_message(tk,reply)       # 回傳訊息
        
    except:
        print('error')
        

    return 'OK'

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\user\AppData\Local\Temp\ipykernel_23196\2733888729.py:18: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token'))
C:\Users\user\AppData\Local\Temp\ipykernel_23196\2733888729.py:19: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(config.get('line-bot', 'channel_secret'))
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


{
    "destination": "Uc867fea521a6b3d28dfdb2d93b7209c0",
    "events": [
        {
            "type": "message",
            "message": {
                "type": "sticker",
                "id": "508505924600594705",
                "quoteToken": "epWyzgsczZHn208SU1Y31WMaLDkkZiCcmjEVJgfGIA6O8L46VVOSqn4-UH3pTu4yE-nDP-KhMqC2g4l8I_pm6Z5uUaUNm3ChHto6xTmjMukqs4hiZnQNrBycE7eyUVgR_wqA8pUskN3TxEfGEODMHg",
                "stickerId": "352563777",
                "packageId": "13361139",
                "stickerResourceType": "STATIC",
                "keywords": [
                    "Exactly",
                    "Hmm",
                    "oh"
                ]
            },
            "webhookEventId": "01HY2H2JQ5Q8D2CC26Z2E87HV1",
            "deliveryContext": {
                "isRedelivery": false
            },
            "timestamp": 1715924454035,
            "source": {
                "type": "user",
                "userId": "U8d9a03f570a9c723ecc43cd7c1654ae8"
            },


C:\Users\user\AppData\Local\Temp\ipykernel_23196\2733888729.py:180: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(tk,sticker_message)
127.0.0.1 - - [17/May/2024 13:40:55] "POST / HTTP/1.1" 200 -


{
    "destination": "Uc867fea521a6b3d28dfdb2d93b7209c0",
    "events": [
        {
            "type": "message",
            "message": {
                "type": "sticker",
                "id": "508505932117574086",
                "quoteToken": "VAp3ibsxzDwqvFmROft12gRa2nbx7GNirekoMvmwizf-Oi9qU_8xrjsfzcxdiVaSXb4DXkk4KiQ31VunZ5SUIYeNRf4jCt2E8Zh9_jJOBohCfbyhHxeN-JFuH4uA4baZRQBN5moXOIoAMqwjL1-fuA",
                "stickerId": "352563777",
                "packageId": "13361139",
                "stickerResourceType": "STATIC",
                "keywords": [
                    "Hmm",
                    "oh",
                    "Exactly"
                ]
            },
            "webhookEventId": "01HY2H2QHH7RJBDX8J2HHXC21K",
            "deliveryContext": {
                "isRedelivery": false
            },
            "timestamp": 1715924458644,
            "source": {
                "type": "user",
                "userId": "U8d9a03f570a9c723ecc43cd7c1654ae8"
            },


...

127.0.0.1 - - [17/May/2024 13:41:28] "POST / HTTP/1.1" 200 -


{
    "destination": "Uc867fea521a6b3d28dfdb2d93b7209c0",
    "events": [
        {
            "type": "message",
            "message": {
                "type": "text",
                "id": "508506003805831393",
                "quoteToken": "57-EUKUIaFgZukYRp2cgyIVW2ZSZgGlQEVvRBUeRDpAjxlEw6oAe5HdnxbSW9tQwzoH3bZ1ZsDVGcxqoECNDWyuQjxJDneXTj7HDUCGfkT6k9tyG1aqGBOgmJvP-276aHYF2atyZibLYgdCCIcF52g",
                "text": "\u6211\u60f3\u8ddf\u4f60\u804a\u5929~"
            },
            "webhookEventId": "01HY2H40XGA1WPFQ6EJ4P7VY12",
            "deliveryContext": {
                "isRedelivery": false
            },
            "timestamp": 1715924501242,
            "source": {
                "type": "user",
                "userId": "U8d9a03f570a9c723ecc43cd7c1654ae8"
            },
            "replyToken": "6febf78b16ca42ff8f3816be861db2d5",
            "mode": "active"
        }
    ]
}
{'channel': 1, 'chat': None, 'foodRecord': None, 'translate_en': ChatSession(
    model=genai

127.0.0.1 - - [17/May/2024 13:41:44] "POST / HTTP/1.1" 200 -


{
    "destination": "Uc867fea521a6b3d28dfdb2d93b7209c0",
    "events": [
        {
            "type": "message",
            "message": {
                "type": "text",
                "id": "508506017496039858",
                "quoteToken": "_BfCSa6ATBT2oXdQfDKP15YIebMtHmRjK1sYm762sGzs8xNBs09_AtX1t5muS9fA-faZ-eimzapCBySVyuEP8DlDeAEsk78FEN3_CA0XpZZ_ABlkLZfbDd0AfhfQthFyH_dCMm4kt4wDBNX8H-dtuA",
                "text": "\u8ddf\u6211\u8aaa\u500b\u7b11\u8a71"
            },
            "webhookEventId": "01HY2H48YZ98KY8Y83EF4N57CX",
            "deliveryContext": {
                "isRedelivery": false
            },
            "timestamp": 1715924509552,
            "source": {
                "type": "user",
                "userId": "U8d9a03f570a9c723ecc43cd7c1654ae8"
            },
            "replyToken": "9c13ca5e937c47cb9a2ff2b5a3da1429",
            "mode": "active"
        }
    ]
}
{'channel': 0, 'chat': ChatSession(
    model=genai.GenerativeModel(
        model_name='model

127.0.0.1 - - [17/May/2024 13:41:53] "POST / HTTP/1.1" 200 -
